In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.6/339.6 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 3.0 MB/s eta 0:00:00


In [4]:
from crewai import Agent, Task, Crew

In [5]:
!pip install openai

In [6]:
!pip install python-dotenv

In [7]:
!pip install openai tiktoken


In [9]:
import os
from dotenv import load_dotenv
from crewai_tools import FileReadTool, ScrapeWebsiteTool, MDXSearchTool, SerperDevTool
from IPython.display import Markdown, display
from langchain_groq import ChatGroq

env_path = '/content/drive/MyDrive/resume_editor/.env'
load_dotenv(env_path)
openai_api_key = ChatGroq(
            api_key=os.getenv("GROQ_API_KEY"),
            model="llama3-70b-8192",
            temperature=0
        )
serper_api_key = os.getenv('SERPER_API_KEY')

In [10]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool(api_key=serper_api_key)
scrape_tool = ScrapeWebsiteTool()

def read_binary_file(file_path):
    try:
        with open(file_path, 'rb') as file:
            return file.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None

# Correct file path
resume_file_path = '/content/drive/MyDrive/resume_editor/Zeba_AI_RESUME`.pdf'

# Read the binary content of the file
binary_content = read_binary_file(resume_file_path)
if binary_content is None:
    raise FileNotFoundError(f"File not found: {resume_file_path}")

# Initialize file reading tools
read_resume = FileReadTool(file_content=binary_content)
semantic_search_resume = MDXSearchTool(mdx_content=binary_content)

# Display a message for the binary data processing
display(Markdown("File loaded and processed. Display not possible directly for binary data."))


File loaded and processed. Display not possible directly for binary data.

In [11]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [12]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [13]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [14]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

Tasks


In [15]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key Responsibilities , skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)


In [16]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using personal write-up "
        "({personal_writeup}) and LinkedIn profile ({linked_in_url}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [17]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from previous tasks, tailor the resume to highlight the most relevant areas. "
        "Employ tools to adjust and enhance the resume content, ensuring the tailored resume aligns perfectly with the job description. "
        "Update every section, including the initial summary, work experience, skills, and education with points from the job posting to better reflect the candidate's abilities "
        "on how they match the job posting. Maintain sections for contact information, education, projects, and certificates, but prioritize "
        "and highlight details that directly match the job description. Use strong action verbs and quantifiable results when describing projects. "
        "Highlight relevant coursework or educational achievements that align with the job requirements. Include only the most relevant certificates "
        "that showcase skills needed for the position. Suggest adding a skills section to the tailored resume if the existing resume doesn't have one, "
        "and populate it with the skills most relevant to the job description."

    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [18]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

In [19]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

In [20]:
job_application_inputs =  {
    'job_posting_url': 'https://www.enfintechnologies.com/vacancies/business-analyst/',
    'linked_in_url': 'https://in.linkedin.com/in/zebasudheer',
    'personal_writeup': """Zeba is a Mechanical Production Engineer familiar with management subjects with passion for technology and programming.Having work experience as an AI engineer and training in full stack development , data analysis and data science. Eager to apply my education, training, and skills to drive innovation and contribute to the success of an organization. Committed to continuous learning and growth, with a goal of building and leading high-performing teams."""
}

In [21]:
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://www.enfintechnologies.com/vacancies/business-analyst/) to extract key Responsibilities , skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using personal write-up (Zeba is a Mechanical Production Engineer familiar with management subjects with passion for technology and programming.Having work experience as an AI engineer and training in full stack development , data analysis and data science. Eager to apply my education, training, and skills to drive innovation and contribute to the success of an organization. Committed to continuous learning and growth, with a goal of building and leading high-perform

In [22]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

Career Summary:
A dynamic and results-driven professional with a strong background in AI software engineering and full-stack development. Accomplished in applying Python, Machine Learning, AI, and data analysis skills to deliver innovative solutions and drive business success. Proven ability to analyse and interpret complex data, identify trends and patterns, and translate findings into actionable business insights. Exceptional communication skills, with a track record of working effectively with diverse teams and key decision makers. Aspiring to leverage these skills and experiences to excel as a Business Analyst.

Work Experience:

1. AI Software Engineer Trainee at MindSpace AI (Jan 2024 - Present)
   - Utilized AI to streamline requirements gathering and communication processes, enhancing efficiency and accuracy.
   - Developed and deployed a chatbot using Natural Language Processing (NLP) techniques, demonstrating strong problem-solving skills.
   - Leveraged Python and Machine Learning for the development of a recommendation system, showcasing ability to translate complex data into actionable insights.
   - Worked closely with diverse teams and key decision makers, enhancing interpersonal and communication skills.

2. Full-stack Developer and Data Scientist at Faith Infotech Academy (Jul 2023 - Jan 2024)
   - Leveraged Full-stack development skills to manage and execute projects, demonstrating strong project management and organizational skills.
   - Utilized Python, Machine Learning, and AI for data analysis, showcasing strong analytical abilities and understanding of business processes.

3. Engineering Intern at KELTRON ADVANCED STUDIES (Dec 2021 - Jan 2022)
   - Gained initial exposure to the IT industry, developing foundational understanding of business processes and systems.

Education:

- Bachelor of Technology - BTech in Mechanical Production Engineering with Computer Science Minor from Sree Chitra Thirunal College of Engineering, Pappanamcode, Thiruvananthapuram (2019 - 2023)
  - Relevant coursework: Data Analysis, Machine Learning, AI

Licenses & Certifications:

- Certificate course in coding skills from ICT Academy of Kerala (Issued Mar 2023)
  - Gained proficiency in Python, a key skill for Business Analysts in IT companies.
- Deep Learning from NPTEL (Issued Jan 2023)
  - Acquired knowledge in Deep Learning, enhancing ability to handle complex data and provide valuable insights.
- Introduction to Machine Learning from NPTEL (Issued Jan 2023)
  - Developed understanding of Machine Learning, a relevant skill for analysing and interpreting data.

Projects:

- Fake news prediction (Aug 2022 - Present)
  Implemented supervised learning using algorithms including Random Forest, Logistic Regression, Decision Tree, and Gradient Boosting, achieving high accuracies to distinguish genuine from fabricated news articles. 
  - This project demonstrates Zeba's ability to handle complex data and provide actionable insights, which is crucial for a Business Analyst role.

Interests:

- Continual learning and growth: Keen to keep abreast of the latest trends and developments in the field of Business Analysis.
- Leading high-performing teams: Proficient in team collaboration and can lead teams to achieve business goals.
- Data Science: Interested in utilising data science techniques to analyse business data and provide actionable insights.
- Full Stack Development: Passionate about leveraging full-stack development skills to manage and execute projects.
- AI Engineering: Interested in applying AI to solve complex business problems.
- Machine Learning: Keen on harnessing Machine Learning for efficient data analysis.

In [23]:
display(Markdown("./interview_materials.md"))

**Interview Questions:**

1. Can you describe a time when you utilized Python, Machine Learning, or AI to deliver a solution to a complex business problem?
2. How have your experiences as an AI Software Engineer Trainee and Full-stack Developer prepared you for the role of a Business Analyst?
3. Can you discuss a project where you used NLP techniques or Machine Learning algorithms for data analysis or problem-solving?
4. The role requires preparing BA artifacts like Proposals, SRS, BRDs, Prototypes etc. Can you provide an example of when you had to prepare similar documents and what approach you took?
5. This role requires interacting with US and European clients across different time zones. How comfortable are you with working across different time zones and how would you manage your time?
6. Can you describe a situation where you had to work with key decision makers and all business levels? How did you handle it?
7. Can you discuss your project on fake news prediction, and how the skills you used there could be applicable to the Business Analyst role?

**Talking Points:**

1. Discuss the development and deployment of the chatbot at MindSpace AI, focusing on how you used AI and NLP techniques for intent classification and response generation.
2. Highlight your ability to analyze and interpret complex data, as demonstrated in your work at Faith Infotech Academy and your fake news prediction project.
3. Explain how your education, particularly your Computer Science Minor and related coursework, has given you a foundation for understanding business processes and systems.
4. Discuss your certifications in coding skills, Deep Learning, and Machine Learning, and how these skills can be applied to the Business Analyst role.
5. Highlight your ability to work effectively with diverse teams and key decision makers, as demonstrated in your AI Software Engineer Trainee role.
6. Discuss your interest in continual learning and growth, leading high-performing teams, and leveraging Data Science, Full Stack Development, AI Engineering, and Machine Learning in the Business Analyst role.

In [24]:
!pip install nbconvert
!apt-get install -y pandoc


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pandoc is already the newest version (2.9.2.1-3ubuntu2).
pandoc set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [25]:
!apt-get update
!apt-get install -y texlive texlive-latex-extra texlive-xetex pandoc


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,377 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,129 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/univers

In [26]:
!pandoc tailored_resume.md -V geometry:margin=1in -o new_resume.pdf

In [27]:
from google.colab import files

files.download('new_resume.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>